# Partie 1 : Lecture et nettoyage des données

## Import librairies

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Lecture des données

In [72]:
dataset = pd.read_csv('./src/Airplane_Crashes_and_Fatalities.csv', encoding='utf-8')
dataset.shape

(4998, 16)

In [73]:
dataset.head()

,Date,Time,Location,Operator,Route,AC Type,Military,Postal_Cargo,Aboard,Aboard Passangers,Aboard Crew,Fatalities,Fatalities Passangers,Fatalities Crew,Ground,Summary
0,9/17/1908,17:18,"Fort Myer, Virginia, USA",Military - U.S. Army,Demonstration,Wright Flyer III,Yes,No,2.0,1.0,1.0,1.0,1.0,0.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,09/07/1909,NaN,"Juvisy-sur-Orge, France",NaN,Air show,Wright Byplane,No,No,1.0,0.0,1.0,1.0,0.0,0.0,0.0,Eugene Lefebvre was the first pilot to ever be...
2,07/12/1912,06:30,"Atlantic City, New Jersey, USA",Military - U.S. Navy,Test flight,Dirigible,Yes,No,5.0,0.0,5.0,5.0,0.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
3,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,NaN,Curtiss seaplane,No,No,1.0,0.0,1.0,1.0,0.0,1.0,0.0,The first fatal airplane accident in Canada oc...
4,09/09/1913,18:30,"Over the North Sea, North Sea",Military - German Navy,NaN,Zeppelin L-1 (airship),Yes,No,20.0,NaN,NaN,14.0,NaN,NaN,0.0,The airship flew into a thunderstorm and encou...


In [74]:
dataset.dtypes

Date                      object
Time                      object
Location                  object
Operator                  object
Route                     object
AC Type                   object
Military                  object
Postal_Cargo              object
Aboard                   float64
Aboard Passangers        float64
Aboard Crew              float64
Fatalities               float64
Fatalities Passangers    float64
Fatalities Crew          float64
Ground                   float64
Summary                   object
dtype: object

## Nettoyage des données
### Valeurs manquantes

In [75]:
# Pourcentage de valeurs manquantes :
dataset.isna().sum().sort_values(ascending=False)/len(dataset)*100

Time                     30.312125
Route                    15.566226
Fatalities Passangers     4.821929
Fatalities Crew           4.801921
Aboard Passangers         4.561825
Aboard Crew               4.501801
Summary                   1.280512
Ground                    0.840336
Aboard                    0.360144
AC Type                   0.300120
Operator                  0.200080
Fatalities                0.160064
Location                  0.100040
Date                      0.000000
Military                  0.000000
Postal_Cargo              0.000000
dtype: float64

### Format des dates (Date, Time)

In [76]:
# Les champs Date et Time sont au format string :
print(type(dataset.loc[0, 'Date']))
print(type(dataset.loc[0, 'Time']))

<class 'str'>
<class 'str'>


In [77]:
# Ajout des champs datetime et time au format date :
from datetime import datetime
from datetime import time

date_format = '%m/%d/%Y'
dataset['datetime'] = dataset['Date'].apply(lambda x: datetime.strptime(x, date_format))

dataset['time'] = dataset['Time'].apply(lambda x: 0 if x != x 
                                            else time(int(x.split(':')[0]), int(x.split(':')[1]), 0))

dataset['hour'] = dataset['Time'].apply(lambda x: np.nan if x != x 
                                            else int(x.split(':')[0]))
                                        
dataset['month'] = dataset['Date'].apply(lambda x: int(x.split('/')[0]))

### Itinéraire (champ Route)

In [78]:
dataset['Route'].value_counts().head(15)

Route
Training                      100
Sightseeing                    39
Test flight                    33
Military exercise               8
Sao Paulo - Rio de Janeiro      7
Demonstration                   6
Rio de Janeiro - Sao Paulo      5
Barranquilla - Bogota           4
Paris - London                  4
Tallinn - Helsinki              4
Huambo - Luanda                 4
Sao Paulo - Porto Alegre        4
Bogota - Barranquilla           4
Survey flight                   4
Villavicencio - Mitu            4
Name: count, dtype: int64

Certains vols sont directs, d'autres effectuent des arrêts.  

Par exemple le vol ci-dessous présente 3 villes dans son itinéraire, il effectue 1 stop à Bishkek.

In [79]:
# Exemple (ligne 4927) Ici l'itinéraire comporte 3 villes :
print(dataset.loc[4927, 'Route'])
print(dataset.loc[4927, 'Route'].split('-'))
print(len(dataset.loc[4927, 'Route'].split('-')))

Hong Kong - Bishkek - Istanbul
['Hong Kong ', ' Bishkek ', ' Istanbul']
3


In [80]:
# Vols non-directs :
non_direct = []
direct = []

for i in range(len(dataset)):
    if dataset.loc[i, 'Route'] == dataset.loc[i, 'Route']:
        temp = dataset.loc[i, 'Route'].split('-')
        if len(temp) > 2 : 
            non_direct.append(i)
        else :
            direct.append(i)

non_direct # indices des vols non-directs
direct # indices des vols directs
print('Vols directs :', len(direct)) 
print('Vols non-directs :', len(non_direct)) # le nombre de vols non-directs

Vols directs : 3609
Vols non-directs : 611


In [81]:
# Nombre de stops lorsque le vol n'est pas direct :
nb_stops = []
for index in non_direct:
    temp = dataset.loc[index, 'Route'].split('-')
    nb_stops.append(len(temp)-2)
set(nb_stops)

{1, 2, 3, 4, 5}

## Enrichissement du dataframe

### Origine et destination

In [82]:
dataset['origin'] = dataset['Route'].apply(lambda x: x.split('-')[0].strip() if (x == x and len(x.split('-')) > 1) else np.nan)

dataset['destination'] = dataset['Route'].apply(lambda x: x.split('-')[-1].strip() if (x == x and len(x.split('-')) > 1) else np.nan)

### Vol direct et nombre de stops

In [83]:
dataset['straight'] = dataset['Route'].apply(lambda x: 'No' if (x == x and len(x.split('-')) > 2)
                                                        else 'Yes' if (x == x and len(x.split('-')) == 2) 
                                                        else np.nan )
                                                        
dataset['stops'] = dataset['Route'].apply(lambda x: len(x.split('-')) - 2 if (x == x and len(x.split('-')) >= 2) else np.nan)

### Pays où a lieu le crash

In [84]:
dataset['country'] = dataset['Location'].apply(lambda x: x.split(',')[-1].strip() if x == x else np.nan)

In [85]:
dataset['country'].value_counts().head(20)

country
USA                 1035
Russia               293
Brazil               176
Colombia             151
Canada               147
France               126
England              110
India                107
Indonesia             99
China                 95
Mexico                89
Germany               89
Italy                 86
Philippines           77
Australia             67
Vietnam               67
Spain                 64
Venezuela             55
Peru                  55
Papua New Guinea      50
Name: count, dtype: int64

## Export csv

In [86]:
# Sauvegarde des données nettoyées dans un fichier csv :
dataset.to_csv('./src/airplane_crashes_clean.csv', index=False)